# Step1 : 安装tushare库

In [1]:
!pip3 install tushare #Mac
# !pip install tushare #Win

# Step2 : 利用个人token调用接口，获取股票基础信息

In [2]:
#导入库
import tushare as t 

#调用接口，后面的字符串是接口，需要使用你自己的TOKEN。获取地：https://tushare.pro
#TOKEN获取：注册账号 -> 头像 -> 个人主页 -> 借口TOKEN
pro =t.pro_api("37e0a2e4efd545f2e312a6c875909fa4e25ddc6ad0d0d0efb3879e1e")

#获取股票基础信息
stock_df = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date') 

In [3]:
#查看基础行值 
stock_df.columns

Index(['ts_code', 'symbol', 'name', 'area', 'industry', 'list_date'], dtype='object')

In [4]:
#存储，避免多次请求
stock_df.to_csv("stock.csv")

In [5]:
#读取数据，获取股票吗
import pandas as pd 
pa = pd.read_csv("stock.csv") 
pa['symbol']

0            1
1            2
2            4
3            5
4            6
         ...  
4800    871857
4801    871981
4802    872925
4803    873169
4804    689009
Name: symbol, Length: 4805, dtype: int64

### 下载对应公司的各项报表

In [6]:
#导入库
import urllib

#设置股票代码
stock_id = "000733"

In [7]:
#下载对应的公司的资产负载表

#构建URL
url = f"http://money.finance.sina.com.cn/corp/go.php/vDOWN_BalanceSheet/displaytype/4/stockid/{stock_id}/ctrl/all.phtml"

#下载文件
urllib.request.urlretrieve(url=url, filename = f"./{stock_id}资产负载表.xls")

('./000733资产负载表.xls', <http.client.HTTPMessage at 0x7fd955aef640>)

In [8]:
#下载利润表

#构建URL
url = f'http://money.finance.sina.com.cn/corp/go.php/vDOWN_ProfitStatement/displaytype/4/stockid/{stock_id}/ctrl/all.phtml'

#下载文件
urllib.request.urlretrieve(url=url, filename = f"./{stock_id}利润表.xls")

('./000733利润表.xls', <http.client.HTTPMessage at 0x7fd955778bb0>)

In [9]:
#下载现金流量表

#构建URL
url = f'http://money.finance.sina.com.cn/corp/go.php/vDOWN_CashFlow/displaytype/4/stockid/{stock_id}/ctrl/all.phtml'

#下载文件
urllib.request.urlretrieve(url=url, filename = f"./{stock_id}现金流量表.xls")

('./000733现金流量表.xls', <http.client.HTTPMessage at 0x7fd952929c70>)

### 处理资产负债表

In [10]:
#读取下载下来的资产负载表数据。

#导入Pandas库
import pandas as pd

#获取文件
stock_id = "000733"
a = pd.read_table(f"./{stock_id}资产负载表.xls",encoding = 'GBK') #使用文本读取方式

#重新设置INDEX
Assets_liabilities_data = pd.DataFrame(a).set_index('报表日期') #设置index

#获取index方便后面检索。
Assets_liabilities_data.index

Index(['单位', '流动资产', '货币资金', '交易性金融资产', '衍生金融资产', '应收票据及应收账款', '应收票据', '应收账款',
       '应收款项融资', '预付款项', '其他应收款(合计)', '应收利息', '应收股利', '其他应收款', '买入返售金融资产',
       '存货', '划分为持有待售的资产', '一年内到期的非流动资产', '待摊费用', '待处理流动资产损益', '其他流动资产',
       '流动资产合计', '非流动资产', '发放贷款及垫款', '可供出售金融资产', '持有至到期投资', '长期应收款', '长期股权投资',
       '投资性房地产', '在建工程(合计)', '在建工程', '工程物资', '固定资产及清理(合计)', '固定资产净额', '固定资产清理',
       '生产性生物资产', '公益性生物资产', '油气资产', '使用权资产', '无形资产', '开发支出', '商誉', '长期待摊费用',
       '递延所得税资产', '其他非流动资产', '非流动资产合计', '资产总计', '流动负债', '短期借款', '交易性金融负债',
       '应付票据及应付账款', '应付票据', '应付账款', '预收款项', '应付手续费及佣金', '应付职工薪酬', '应交税费',
       '其他应付款(合计)', '应付利息', '应付股利', '其他应付款', '预提费用', '一年内的递延收益', '应付短期债券',
       '一年内到期的非流动负债', '其他流动负债', '流动负债合计', '非流动负债', '长期借款', '应付债券', '租赁负债',
       '长期应付职工薪酬', '长期应付款(合计)', '长期应付款', '专项应付款', '预计非流动负债', '递延所得税负债',
       '长期递延收益', '其他非流动负债', '非流动负债合计', '负债合计', '所有者权益', '实收资本(或股本)', '资本公积',
       '减：库存股', '其他综合收益', '专项储备', '盈余公积', '一般风险准备', '未分配利润', '归属于母公司股东权益合计',
       '少数股

In [11]:
#清洁数据

#去除无效值
Assets_liabilities_data.replace('--', 0, inplace = True) 

#去除NaN值
Assets_liabilities_data.replace(pd.NA,0,inplace=True) 

#去除无意义行
Assets_liabilities_data = Assets_liabilities_data.drop(index = '单位') 

#转换数据为浮点数
Assets_liabilities_data = Assets_liabilities_data.applymap(lambda x: float(x)) 

#验证前五行
Assets_liabilities_data.head() 

,20220331,20211231,20210930,20210630,20210331,20201231,20200930,20200630,20200331,20191231,...,19990630,19981231,19980630,19971231,19970630,19961231,19951231,19941231,19700101,Unnamed: 96
报表日期,,,,,,,,,,,,,,,,,,,,,
流动资产,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.00,0.00,0.00,NaN,NaN
货币资金,1.305131e+09,1.647893e+09,1.054360e+09,1.122313e+09,7.749220e+08,9.176935e+08,8.276289e+08,8.807181e+08,1.014840e+09,1.341113e+09,...,2.092787e+08,84674578.8,1.432936e+08,2.684448e+08,0.0,65476548.27,40907534.66,32553634.16,0.0,NaN
交易性金融资产,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.400001e+08,0.0,1.300000e+08,1.300000e+08,0.0,0.00,0.00,15000.00,0.0,NaN
衍生金融资产,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.00,0.00,0.00,0.0,NaN
应收票据及应收账款,4.884096e+09,3.861235e+09,4.437501e+09,4.174164e+09,3.972537e+09,3.418209e+09,3.891195e+09,3.699396e+09,3.256845e+09,2.828699e+09,...,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.00,0.00,0.00,0.0,NaN


### 处理现金流量数据

In [12]:
#读取下载下来的现金流量数据。

#读取文件，使用文本读取方式
stock_id = "000733"
a = pd.read_table(f"./{stock_id}现金流量表.xls",encoding = 'GBK')

#设置index
CashFlow = pd.DataFrame(a).set_index('报表日期') 

#获取index方便后面检索。
CashFlow.index

Index(['单位', '一、经营活动产生的现金流量', '销售商品、提供劳务收到的现金', '收到的税费返还', '收到的其他与经营活动有关的现金',
       '经营活动现金流入小计', '购买商品、接受劳务支付的现金', '支付给职工以及为职工支付的现金', '支付的各项税费',
       '支付的其他与经营活动有关的现金', '经营活动现金流出小计', '经营活动产生的现金流量净额', '二、投资活动产生的现金流量',
       '收回投资所收到的现金', '取得投资收益所收到的现金', '处置固定资产、无形资产和其他长期资产所收回的现金净额',
       '处置子公司及其他营业单位收到的现金净额', '收到的其他与投资活动有关的现金', '投资活动现金流入小计',
       '购建固定资产、无形资产和其他长期资产所支付的现金', '投资所支付的现金', '取得子公司及其他营业单位支付的现金净额',
       '支付的其他与投资活动有关的现金', '投资活动现金流出小计', '投资活动产生的现金流量净额', '三、筹资活动产生的现金流量',
       '吸收投资收到的现金', '其中：子公司吸收少数股东投资收到的现金', '取得借款收到的现金', '发行债券收到的现金',
       '收到其他与筹资活动有关的现金', '筹资活动现金流入小计', '偿还债务支付的现金', '分配股利、利润或偿付利息所支付的现金',
       '其中：子公司支付给少数股东的股利、利润', '支付其他与筹资活动有关的现金', '筹资活动现金流出小计', '筹资活动产生的现金流量净额',
       '四、汇率变动对现金及现金等价物的影响', '五、现金及现金等价物净增加额', '加:期初现金及现金等价物余额',
       '六、期末现金及现金等价物余额', '附注', '净利润', '少数股东权益', '未确认的投资损失', '资产减值准备',
       '固定资产折旧、油气资产折耗、生产性物资折旧', '无形资产摊销', '长期待摊费用摊销', '待摊费用的减少', '预提费用的增加',
       '处置固定资产、无形资产和其他长期资产的损失', '固定资产报废损失', '公允价值变动损失', '递延收益增加（减：减少）', 

In [13]:
#清洁数据

#去除无效值
CashFlow.replace('--', 0, inplace = True) 

#去除NaN值
CashFlow.replace(pd.NA,0,inplace=True) 

#去除无意义行
CashFlow = CashFlow.drop(index = '单位') 

#转换数据为浮点数
CashFlow = CashFlow.applymap(lambda x : float(x)) 

#查看前五行
CashFlow.head() 

,20220331,20211231,20210930,20210630,20210331,20201231,20200930,20200630,20200331,20191231,...,20020630,20011231,20010630,20001231,20000630,19991231,19981231,19980630,19700101,Unnamed: 88
报表日期,,,,,,,,,,,,,,,,,,,,,
一、经营活动产生的现金流量,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
销售商品、提供劳务收到的现金,1.069493e+09,5.419928e+09,3.403944e+09,2.186274e+09,8.383749e+08,3.017134e+09,1.780196e+09,1.152107e+09,4.983867e+08,3.446931e+09,...,289481000.0,6.925827e+08,2.706763e+08,7.472772e+08,3.518864e+08,5.691690e+08,5.727557e+08,1.582246e+08,0.0,NaN
收到的税费返还,9.789547e+05,6.685491e+06,6.645438e+06,3.856668e+06,8.613060e+03,3.354553e+07,3.143955e+07,3.137906e+07,0.000000e+00,2.209035e+07,...,1192000.0,1.101257e+07,1.299867e+06,5.000000e+04,6.714578e+05,7.330854e+05,0.000000e+00,2.287080e+07,0.0,NaN
收到的其他与经营活动有关的现金,1.279864e+08,3.838492e+08,3.005921e+08,1.963162e+08,1.058835e+08,5.483149e+08,4.316828e+08,3.133228e+08,3.906732e+07,4.033195e+08,...,49611000.0,1.022514e+08,7.517343e+07,1.542238e+08,1.124324e+08,1.243420e+08,4.058066e+07,1.730899e+07,0.0,NaN
经营活动现金流入小计,1.198459e+09,5.810462e+09,3.711181e+09,2.386447e+09,9.442669e+08,3.598995e+09,2.243319e+09,1.496809e+09,5.374540e+08,3.872341e+09,...,340284000.0,8.058466e+08,3.471496e+08,9.015510e+08,4.649903e+08,6.942441e+08,6.133364e+08,1.984044e+08,0.0,NaN


### 处理利润数据

In [14]:
#读取下载下来的利润数据

#读取文件，使用文本读取方式
stock_id = "000733"
a = pd.read_table(f"./{stock_id}利润表.xls",encoding = 'GBK') 

#设置index
ProfitStatement = pd.DataFrame(a).set_index('报表日期') 

#获取index方便后面检索。
ProfitStatement.index 

Index(['单位', '一、营业总收入', '营业收入', '二、营业总成本', '营业成本', '营业税金及附加', '销售费用', '管理费用',
       '财务费用', '研发费用', '资产减值损失', '公允价值变动收益', '投资收益', '其中:对联营企业和合营企业的投资收益',
       '汇兑收益', '三、营业利润', '加:营业外收入', '减：营业外支出', '其中：非流动资产处置损失', '四、利润总额',
       '减：所得税费用', '五、净利润', '归属于母公司所有者的净利润', '少数股东损益', '六、每股收益', '基本每股收益(元/股)',
       '稀释每股收益(元/股)', '七、其他综合收益', '八、综合收益总额', '归属于母公司所有者的综合收益总额',
       '归属于少数股东的综合收益总额'],
      dtype='object', name='报表日期')

In [15]:
#清洁数据

#去除无效值
ProfitStatement.replace('--', 0, inplace = True) 

#去除NaN值
ProfitStatement.replace(pd.NA,0,inplace=True) 

#去除无意义行
ProfitStatement = ProfitStatement.drop(index = '单位') 

#转换数据为浮点数
ProfitStatement = ProfitStatement.applymap(lambda x : float(x)) 

#查看前五行
ProfitStatement.head() 

,20220331,20211231,20210930,20210630,20210331,20201231,20200930,20200630,20200331,20191231,...,19990630,19981231,19980630,19971231,19970630,19961231,19951231,19941231,19700101,Unnamed: 96
报表日期,,,,,,,,,,,,,,,,,,,,,
一、营业总收入,1.885912e+09,5.655972e+09,4.248034e+09,2.816665e+09,1.308220e+09,3.949731e+09,3.071142e+09,2.024859e+09,8.884147e+08,3.668282e+09,...,2.633305e+08,5.160197e+08,2.665170e+08,4.905803e+08,1.878599e+08,4.156094e+08,4.069253e+08,3.181901e+08,0.0,NaN
营业收入,1.885912e+09,5.655972e+09,4.248034e+09,2.816665e+09,1.308220e+09,3.949731e+09,3.071142e+09,2.024859e+09,8.884147e+08,3.668282e+09,...,2.617253e+08,5.119683e+08,2.658869e+08,4.872204e+08,1.863812e+08,4.135750e+08,4.066341e+08,3.176003e+08,0.0,NaN
二、营业总成本,1.114915e+09,3.891856e+09,3.082351e+09,2.161585e+09,9.873309e+08,3.051154e+09,2.412979e+09,1.601214e+09,7.385145e+08,3.101721e+09,...,2.329584e+08,4.255914e+08,2.212456e+08,3.991801e+08,0.000000e+00,3.531099e+08,3.499032e+08,2.694055e+08,0.0,NaN
营业成本,7.004566e+08,2.216281e+09,1.773529e+09,1.221382e+09,6.062722e+08,1.833802e+09,1.466768e+09,9.437631e+08,4.342628e+08,2.040441e+09,...,1.844978e+08,3.446707e+08,1.833214e+08,3.120141e+08,0.000000e+00,2.672747e+08,2.740207e+08,2.104326e+08,0.0,NaN
营业税金及附加,1.763523e+07,5.071542e+07,3.693921e+07,2.400994e+07,1.206703e+07,3.998452e+07,2.904088e+07,1.912139e+07,8.824359e+06,3.573169e+07,...,1.768570e+06,2.127313e+06,9.178816e+05,2.349085e+06,0.000000e+00,1.793827e+06,1.280107e+06,9.660397e+05,0.0,NaN


# Step3 : 按照公式计算关键指标

## Part3.1 : 盈利能力

### Including ：销售利润率｜总资产净利润率｜权益资本利润率

#### 销售利润率 = 净利润/销售收入

#### 总资产净利润率=净利润/平均总资产

#### 权益资本利润率(ROE)=净利润/平均权益资本

In [16]:
#创建一个空DataFrame存储数据
KPI = pd.DataFrame()

#计算：销售利润率=净利润/销售收入
KPI['销售利润率'] = ProfitStatement.loc['五、净利润'] / ProfitStatement.loc['一、营业总收入']

#计算：总资产净利润率=净利润/平均总资产
KPI['总资产净利润率'] = ProfitStatement.loc['五、净利润'] / (Assets_liabilities_data.loc['资产总计'] + Assets_liabilities_data.loc['资产总计'].shift(-1))*2

#计算：权益资本利润率(ROE)=净利润/平均权益资本
KPI['权益资本利润率'] = ProfitStatement.loc['五、净利润'] / (Assets_liabilities_data.loc['所有者权益(或股东权益)合计'] + Assets_liabilities_data.loc['所有者权益(或股东权益)合计'].shift(-1))*2

KPI

,销售利润率,总资产净利润率,权益资本利润率
20220331,0.322053,0.052742,0.078436
20211231,0.264670,0.136714,0.209537
20210930,0.226386,0.091991,0.144929
20210630,0.183908,0.052860,0.081895
20210331,0.189347,0.026888,0.040599
...,...,...,...
19961231,0.152439,0.111429,0.235712
19951231,0.152947,0.141405,0.444808
19941231,0.149662,0.240750,0.709827
19700101,NaN,NaN,NaN


## Part3.2 : 流动性的指标

### Including : 流动比率|速动比率|现金比率

#### 流动比率=流动资产/流动负债

#### 速动比率=(流动资产-存货)/流动负债

#### 现金比率=货币资金/流动负债

In [17]:

#计算：流动比率=流动资产/流动负债
KPI['流动比率'] = Assets_liabilities_data.loc['流动资产合计'] / Assets_liabilities_data.loc['流动负债合计']

#计算：速动比率=(流动资产-存货)/流动负债
KPI['速动比率'] = (Assets_liabilities_data.loc['流动资产合计']-Assets_liabilities_data.loc['存货']) /Assets_liabilities_data.loc['流动负债合计']

#计算：现金比率=货币资金/流动负债
KPI['现金比率'] = Assets_liabilities_data.loc['货币资金']/Assets_liabilities_data.loc['流动负债合计']

KPI

,销售利润率,总资产净利润率,权益资本利润率,流动比率,速动比率,现金比率
20220331,0.322053,0.052742,0.078436,3.057660,2.368033,0.457079
20211231,0.264670,0.136714,0.209537,2.960863,2.275562,0.611713
20210930,0.226386,0.091991,0.144929,2.698071,2.135891,0.369167
20210630,0.183908,0.052860,0.081895,2.678321,2.176599,0.421068
20210331,0.189347,0.026888,0.040599,2.825292,2.309175,0.337068
...,...,...,...,...,...,...
19961231,0.152439,0.111429,0.235712,1.595486,1.119004,0.218343
19951231,0.152947,0.141405,0.444808,1.250791,0.773362,0.153082
19941231,0.149662,0.240750,0.709827,1.395725,0.826915,0.154746
19700101,NaN,NaN,NaN,NaN,NaN,NaN


## Part3.3 : 偿债能力和资产营运能力

### Including : 总资产负债率|总资产周转次数

#### 总资产负债率=总负债/总资产

#### 总资产周转次数=销售收入/平均总资产

In [18]:
#计算：总资产负债率=总负债/总资产
KPI['总资产负债率'] = Assets_liabilities_data.loc['负债合计'] / Assets_liabilities_data.loc['资产总计']

#计算：总资产周转次数=销售收入/平均总资产
KPI['总资产周转次数'] = ProfitStatement.loc['一、营业总收入'] / (Assets_liabilities_data.loc['资产总计']+Assets_liabilities_data.loc['资产总计'].shift(-1))*2

KPI

,销售利润率,总资产净利润率,权益资本利润率,流动比率,速动比率,现金比率,总资产负债率,总资产周转次数
20220331,0.322053,0.052742,0.078436,3.057660,2.368033,0.457079,0.324053,0.163769
20211231,0.264670,0.136714,0.209537,2.960863,2.275562,0.611713,0.331351,0.516545
20210930,0.226386,0.091991,0.144929,2.698071,2.135891,0.369167,0.364254,0.406346
20210630,0.183908,0.052860,0.081895,2.678321,2.176599,0.421068,0.366348,0.287425
20210331,0.189347,0.026888,0.040599,2.825292,2.309175,0.337068,0.341913,0.142002
...,...,...,...,...,...,...,...,...
19961231,0.152439,0.111429,0.235712,1.595486,1.119004,0.218343,0.705889,0.730972
19951231,0.152947,0.141405,0.444808,1.250791,0.773362,0.153082,0.699456,0.924540
19941231,0.149662,0.240750,0.709827,1.395725,0.826915,0.154746,0.660832,1.608629
19700101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part3.4 : 现金创造能力的指标

### Including : 销售获现率 / 净利润的现金含量

#### 销售获现率=经营性现金净流量/销售收入

#### 净利润的现金含量=经营性现金净流量/净利润

In [19]:
#计算：销售获现率=经营性现金净流量/销售收入
KPI['销售获现率'] = CashFlow.loc['经营活动产生的现金流量净额'] / ProfitStatement.loc['一、营业总收入']


#计算：净利润的现金含量=经营性现金净流量/净利润
KPI['净利润的现金含量'] = CashFlow.loc['经营活动产生的现金流量净额'] / ProfitStatement.loc['五、净利润']

KPI

,销售利润率,总资产净利润率,权益资本利润率,流动比率,速动比率,现金比率,总资产负债率,总资产周转次数,销售获现率,净利润的现金含量
20220331,0.322053,0.052742,0.078436,3.057660,2.368033,0.457079,0.324053,0.163769,0.015756,0.048923
20211231,0.264670,0.136714,0.209537,2.960863,2.275562,0.611713,0.331351,0.516545,0.236097,0.892044
20210930,0.226386,0.091991,0.144929,2.698071,2.135891,0.369167,0.364254,0.406346,0.157160,0.694215
20210630,0.183908,0.052860,0.081895,2.678321,2.176599,0.421068,0.366348,0.287425,0.163061,0.886646
20210331,0.189347,0.026888,0.040599,2.825292,2.309175,0.337068,0.341913,0.142002,0.018119,0.095690
...,...,...,...,...,...,...,...,...,...,...
19961231,0.152439,0.111429,0.235712,1.595486,1.119004,0.218343,0.705889,0.730972,NaN,NaN
19951231,0.152947,0.141405,0.444808,1.250791,0.773362,0.153082,0.699456,0.924540,NaN,NaN
19941231,0.149662,0.240750,0.709827,1.395725,0.826915,0.154746,0.660832,1.608629,NaN,NaN
19700101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step4 : 最终数据清理

In [20]:
#替换NAN为0
KPI.fillna(0,inplace=True)

KPI

,销售利润率,总资产净利润率,权益资本利润率,流动比率,速动比率,现金比率,总资产负债率,总资产周转次数,销售获现率,净利润的现金含量
20220331,0.322053,0.052742,0.078436,3.057660,2.368033,0.457079,0.324053,0.163769,0.015756,0.048923
20211231,0.264670,0.136714,0.209537,2.960863,2.275562,0.611713,0.331351,0.516545,0.236097,0.892044
20210930,0.226386,0.091991,0.144929,2.698071,2.135891,0.369167,0.364254,0.406346,0.157160,0.694215
20210630,0.183908,0.052860,0.081895,2.678321,2.176599,0.421068,0.366348,0.287425,0.163061,0.886646
20210331,0.189347,0.026888,0.040599,2.825292,2.309175,0.337068,0.341913,0.142002,0.018119,0.095690
...,...,...,...,...,...,...,...,...,...,...
19961231,0.152439,0.111429,0.235712,1.595486,1.119004,0.218343,0.705889,0.730972,0.000000,0.000000
19951231,0.152947,0.141405,0.444808,1.250791,0.773362,0.153082,0.699456,0.924540,0.000000,0.000000
19941231,0.149662,0.240750,0.709827,1.395725,0.826915,0.154746,0.660832,1.608629,0.000000,0.000000
19700101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
#设置数据为保留两位小数。
pd.options.display.float_format='{:,.2f}'.format 

KPI

,销售利润率,总资产净利润率,权益资本利润率,流动比率,速动比率,现金比率,总资产负债率,总资产周转次数,销售获现率,净利润的现金含量
20220331,0.32,0.05,0.08,3.06,2.37,0.46,0.32,0.16,0.02,0.05
20211231,0.26,0.14,0.21,2.96,2.28,0.61,0.33,0.52,0.24,0.89
20210930,0.23,0.09,0.14,2.70,2.14,0.37,0.36,0.41,0.16,0.69
20210630,0.18,0.05,0.08,2.68,2.18,0.42,0.37,0.29,0.16,0.89
20210331,0.19,0.03,0.04,2.83,2.31,0.34,0.34,0.14,0.02,0.10
...,...,...,...,...,...,...,...,...,...,...
19961231,0.15,0.11,0.24,1.60,1.12,0.22,0.71,0.73,0.00,0.00
19951231,0.15,0.14,0.44,1.25,0.77,0.15,0.70,0.92,0.00,0.00
19941231,0.15,0.24,0.71,1.40,0.83,0.15,0.66,1.61,0.00,0.00
19700101,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [22]:
#反转Index，便于绘图
KPI = KPI.reindex(index=KPI.index[::-1]) 

KPI

,销售利润率,总资产净利润率,权益资本利润率,流动比率,速动比率,现金比率,总资产负债率,总资产周转次数,销售获现率,净利润的现金含量
Unnamed: 96,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
19700101,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
19941231,0.15,0.24,0.71,1.40,0.83,0.15,0.66,1.61,0.00,0.00
19951231,0.15,0.14,0.44,1.25,0.77,0.15,0.70,0.92,0.00,0.00
19961231,0.15,0.11,0.24,1.60,1.12,0.22,0.71,0.73,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
20210331,0.19,0.03,0.04,2.83,2.31,0.34,0.34,0.14,0.02,0.10
20210630,0.18,0.05,0.08,2.68,2.18,0.42,0.37,0.29,0.16,0.89
20210930,0.23,0.09,0.14,2.70,2.14,0.37,0.36,0.41,0.16,0.69
20211231,0.26,0.14,0.21,2.96,2.28,0.61,0.33,0.52,0.24,0.89


In [23]:
#去除无意义行
KPI = KPI.drop(index = 'Unnamed: 96') 

#写入Excel
KPI.to_excel("Result.xlsx") 

KPI

,销售利润率,总资产净利润率,权益资本利润率,流动比率,速动比率,现金比率,总资产负债率,总资产周转次数,销售获现率,净利润的现金含量
19700101,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
19941231,0.15,0.24,0.71,1.40,0.83,0.15,0.66,1.61,0.00,0.00
19951231,0.15,0.14,0.44,1.25,0.77,0.15,0.70,0.92,0.00,0.00
19961231,0.15,0.11,0.24,1.60,1.12,0.22,0.71,0.73,0.00,0.00
19970630,0.17,0.04,0.05,2.89,2.89,0.00,0.33,0.21,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
20210331,0.19,0.03,0.04,2.83,2.31,0.34,0.34,0.14,0.02,0.10
20210630,0.18,0.05,0.08,2.68,2.18,0.42,0.37,0.29,0.16,0.89
20210930,0.23,0.09,0.14,2.70,2.14,0.37,0.36,0.41,0.16,0.69
20211231,0.26,0.14,0.21,2.96,2.28,0.61,0.33,0.52,0.24,0.89
